In [ ]:
# Helper function -- makes all territory plots for England
def make_plots():
    plots = []
    min_lat = 49.97313635
    max_lat = 55.71632200
    min_lon = -5.70440710
    max_lon = 1.723859000
    lat_interval = (max_lat - min_lat)/50
    lon_interval = (max_lon - min_lon)/50
    clat = min_lat
    clon = min_lon
    for i in range(50):
        lat = clat
        clon = min_lon
        for j in range(50):
            lon = clon
            plots.append([(lat, lat+lat_interval), (lon, lon+lon_interval)])
            clon += lon_interval
        clat+= lat_interval
    return plots

In [ ]:
# Helper function -- removes selected plots
def trim(plots, lmax, lmin, latmax, latmin):
    keep = []
    for p in plots:
        lat_min, lat_max = p[0]
        lon_min, lon_max = p[1]
        if lmax >= lon_min >= lmin:
            if latmax >= lat_min >= latmin:
                pass
            else:
                keep.append(p)
        else:
            keep.append(p)
    return keep

In [ ]:
# Helper function -- selects extraneous plots to remove
def trim_plots():
    plots = make_plots()
    a = trim(plots, -3.6, -5.8, 60, 53.4760)
    b = trim(a, 1.7, -1.5, 60, 54.8)
    c = trim(b, 1.7, -0.4, 60, 54.3)
    d = trim(c, 1.7, -0.05, 60, 53.7)
    e = trim(d, 1.7, 0.09, 60, 53.35)
    f = trim(e, 1.7, 0.25, 60, 53)
    g = trim(f, 1.7, 1.1, 51, 49)
    h = trim(g, 1.7, 0.25, 50.7, 49)
    i = trim(h, 1.7, -3.5, 50.4, 49)
    j = trim(i, -4.9, -5.8, 51.48, 50.6)
    k = trim(j, -4.9, -5.8, 60, 52.1)
    l = trim(k, -2.9, -5.8, 60, 55.1)


    return l

In [ ]:
#--------------------------------------------------------------------------------------------------
# GOAL: create 1571 rows (one for each trimmed territory plot) and 550 date columns (100 BC to 450 AD)
#--------------------------------------------------------------------------------------------------
ALL_PLOTS = trim_plots()
PLOT_OBJECTS = []
ERRORS = 0
# make an empty plot object for each trimmed plot
def plot_object(pbounds):
    years = {}
    for i in range((-600), 451):
        years[i] = 0
    # return ((latmin, latmax), (lonmin, lonmax), years_dict)
    return [pbounds[0], pbounds[1], years]

def match_brooch_to_plot(lat, lon):
    found = 0
    for i in range(len(ALL_PLOTS)):
        if found == 0:
            plot = ALL_PLOTS[i]
            if (plot[0][0] <= lat <= plot[0][1]) and (plot[1][0] <= lon <= plot[1][1]):
                # execute
                found = 1
                #print(i)
                return(i)
        else:
            break
    return "ERROR"

def update_plot(index, brooch_n1, brooch_n2):
    value = 1/(abs(brooch_n2 - brooch_n1)+1)
    value_given = 0
    plot = PLOT_OBJECTS[index]
    years = plot[2]
    if brooch_n1 < (-600):
        start = (-600)
    else:
        start = brooch_n1
    if brooch_n2 > 450:
        end = 450
    else:
        end = brooch_n2
        
    for i in range(start, (end + 1)):
        years[i]+=value
        value_given+=value

    plot[2] = years
    PLOT_OBJECTS[index] = plot
    
def evaluate_brooch(lat, lon, n1, n2):
    index = match_brooch_to_plot(lat, lon)
    if index != "ERROR":
        update_plot(index, n1, n2)
        return 0
    else:
        return 1
        
#--------------------------------------------------------------------------------------------------

In [ ]:
# Actually calculate the EV/Territory Plot Heatmap
def CALC_SIG_EVAL_MAP(sig):
    tracker = 0
    ERRORS = 0
    for row in UPDATED_BROOCHES:
        # KEY ROW -----------
        if row['significance'] == sig:
        # -------------------
            lat = row['fourFigureLat']
            lon = row['fourFigureLon']
            n1 = int(row['numdate1'])
            n2 = int(row['numdate2'])
            ERRORS+=evaluate_brooch(lat, lon, n1, n2)
            tracker +=1
    outfile = open('EVals_' + str(sig) + '.csv', 'w')
    writer = csv.writer(outfile)
    cols = ['LAT', 'LON','latmin', 'latmax', 'lonmin', 'lonmax','PLOT_ALLTIME_EV','RATIO','SIG_TOTAL',
           '600_101','100_42','43_65','66_90','91_120','121_150',
            '151_180','181_210','211_250','251_300','301_350','351_450',
           'R600_101','R100_42','R43_65','R66_90','R91_120','R121_150',
            'R151_180','R181_210','R211_250','R251_300','R301_350','R351_450']
    for i in range(-600, 451):
        cols.append(i)
    writer.writerow(cols)
 
    for track in range(len(PLOT_OBJECTS)):
        p = PLOT_OBJECTS[track]
        latmin = p[0][0]
        latmax = p[0][1]
        lonmin = p[1][0]
        lonmax = p[1][1]
        LAT = mean([latmin, latmax])
        LON = mean([lonmin, lonmax])
        YV = p[2]
        yearvals = []
        for i in range(-600, 451):
            add = p[2][i]
            yearvals.append(add)
        
        pcols = [LAT, LON, latmin, latmax, lonmin, lonmax, TOTAL_EVS[track],0]
        TOTAL = sum(yearvals[0:])
        A = sum(yearvals[0:500])
        B = sum(yearvals[500:643])
        C = sum(yearvals[643:666])
        D = sum(yearvals[666:691])
        E = sum(yearvals[691:721])
        F = sum(yearvals[721:751])
        G = sum(yearvals[751:781])
        H = sum(yearvals[781:811])
        I = sum(yearvals[811:851])
        J = sum(yearvals[851:901])
        K = sum(yearvals[901:951])
        L = sum(yearvals[951:])
        for item in [TOTAL,A,B,C,D,E,F,G,H,I,J,K,L]:
            pcols.append(item)
        if TOTAL_EVS[track] == 0:
            pcols[7] = 0
        else:
            pcols[7] = (TOTAL/TOTAL_EVS[track])
        for item in [A,B,C,D,E,F,G,H,I,J,K,L]:
            if TOTAL_EVS[track] == 0:
                pcols.append(0)
            else:
                pcols.append(item/TOTAL_EVS[track])
        for i in range(len(yearvals)):
            pcols.append(yearvals[i])
        writer.writerow(pcols)

    outfile.close()

In [ ]:
# Run the above
PLOT_OBJECTS = []
for p in ALL_PLOTS:
    PLOT_OBJECTS.append(plot_object(p))
CALC_SIG_EVAL_MAP('IMPORT')